In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import eif as iso
import category_encoders as ce

In [2]:
%load_ext sql
%sql sqlite:///Diamonds

In [3]:
db = %sql select * from Diamond
db = db.DataFrame()
db = db.set_index("id")

 * sqlite:///Diamonds
Done.


In [4]:
db[:5]

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,Premium,D,SI2,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,Ideal,E,VVS2,62.7,56.0,6.42,6.46,4.04,9.183
2,0.72,Ideal,F,VS2,61.8,59.0,5.71,5.74,3.54,7.983
3,1.08,Very Good,G,SI2,63.2,57.0,6.54,6.50,4.12,8.371
4,0.36,Premium,G,VS1,62.3,59.0,4.50,4.55,2.82,6.588


In [5]:
db.loc[:,'cut':'clarity'][:5]

,cut,color,clarity
id,,,
0,Premium,D,SI2
1,Ideal,E,VVS2
2,Ideal,F,VS2
3,Very Good,G,SI2
4,Premium,G,VS1


In [6]:
db.describe()

,carat,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.797576,61.746377,57.451694,5.729163,5.732896,3.537351,7.783822
std,0.475939,1.436796,2.234569,1.125244,1.154799,0.711002,1.016828
min,0.200000,43.000000,44.000000,0.000000,0.000000,0.000000,5.787000
25%,0.400000,61.000000,56.000000,4.710000,4.720000,2.910000,6.851000
50%,0.700000,61.800000,57.000000,5.690000,5.700000,3.520000,7.780000
75%,1.040000,62.500000,59.000000,6.540000,6.540000,4.030000,8.581000
max,5.010000,79.000000,95.000000,10.740000,58.900000,31.800000,9.842000


In [7]:
db.isnull().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
x          0
y          0
z          0
price      0
dtype: int64

In [8]:
encoder = ce.TargetEncoder(cols=['cut', 'color', 'clarity'])

/home/user/jupyter_env/lib/python3.8/site-packages/category_encoders/target_encoder.py:122: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter min_samples_leaf will change in version 2.6."
/home/user/jupyter_env/lib/python3.8/site-packages/category_encoders/target_encoder.py:127: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  warnings.warn("Default parameter smoothing will change in version 2.6."


In [9]:
updated_columns = encoder.fit_transform(X=db.loc[:,'cut':'clarity'], y=db['depth'])

In [10]:
db2 = db
db2['cut'] = updated_columns['cut']
db2['color'] = updated_columns['color']
db2['clarity'] = updated_columns['clarity']
db2[:5]

,carat,cut,color,clarity,depth,table,x,y,z,price
id,,,,,,,,,,
0,0.30,61.259779,61.690333,61.762881,62.4,58.0,4.31,4.28,2.68,6.353
1,1.01,61.710573,61.658822,61.663418,62.7,56.0,6.42,6.46,4.04,9.183
2,0.72,61.710573,61.691033,61.714116,61.8,59.0,5.71,5.74,3.54,7.983
3,1.08,61.812751,61.761153,61.762881,63.2,57.0,6.54,6.50,4.12,8.371
4,0.36,61.259779,61.761153,61.680194,62.3,59.0,4.50,4.55,2.82,6.588


In [11]:
x = db2.values
columns = db2.columns

In [12]:
min_max_Scaler = MinMaxScaler()
x_scaled = min_max_Scaler.fit_transform(x)
db2 = pd.DataFrame(x_scaled, columns=columns)
db2[:10]

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.020790,0.000000,0.139749,0.203597,0.538889,0.274510,0.401304,0.072666,0.084277,0.139581
1,0.168399,0.161548,0.000000,0.128862,0.547222,0.235294,0.597765,0.109677,0.127044,0.837485
2,0.108108,0.161548,0.142853,0.166956,0.522222,0.294118,0.531657,0.097453,0.111321,0.541554
3,0.182952,0.198164,0.453828,0.203597,0.561111,0.254902,0.608939,0.110357,0.129560,0.637238
4,0.033264,0.000000,0.453828,0.141467,0.536111,0.294118,0.418994,0.077250,0.088679,0.197534
5,0.068607,0.161548,0.142853,0.141467,0.511111,0.254902,0.482309,0.088285,0.100314,0.421455
6,0.320166,0.000000,0.752440,0.266833,0.530556,0.294118,0.716015,0.129542,0.149686,0.787176
7,0.027027,0.161548,0.000000,0.128862,0.472222,0.235294,0.421788,0.077419,0.085535,0.268804
8,0.024948,0.198164,0.000000,0.166956,0.552778,0.235294,0.405959,0.074533,0.086478,0.189149
9,0.014553,0.161548,0.000000,0.128862,0.522222,0.254902,0.387337,0.069949,0.080503,0.220962


In [13]:
db2.describe()

,carat,cut,color,clarity,depth,table,x,y,z,price
count,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000,40455.000000
mean,0.124236,0.174379,0.388298,0.191197,0.520733,0.263759,0.533442,0.097333,0.111237,0.492435
std,0.098948,0.180446,0.325753,0.114044,0.039911,0.043815,0.104771,0.019606,0.022359,0.250759
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.041580,0.000000,0.139749,0.141467,0.500000,0.235294,0.438547,0.080136,0.091509,0.262392
50%,0.103950,0.161548,0.453828,0.166956,0.522222,0.254902,0.529795,0.096774,0.110692,0.491492
75%,0.174636,0.198164,0.752440,0.266833,0.541667,0.294118,0.608939,0.111036,0.126730,0.689026
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
db_temp = db2[["carat", "depth", "table", "x", "y", "z", "price"]].values

In [15]:
column_eif = iso.iForest(db_temp, ntrees = 500, sample_size = 256, ExtensionLevel = 1)

In [16]:
S1 = column_eif.compute_paths(X_in = db_temp)

In [19]:
int(len(S1) * 0.05)

2022

In [22]:
np.argsort(S1)[-int(len(S1) * 0.05):]

array([21999, 32294, 35750, ..., 22108,  7978,  6403])

In [23]:
mod_data = db2.loc[np.argsort(S1)[:-int(len(S1) * 0.05)]]

In [24]:
db2.std(), mod_data.std()

(carat      0.098948
 cut        0.180446
 color      0.325753
 clarity    0.114044
 depth      0.039911
 table      0.043815
 x          0.104771
 y          0.019606
 z          0.022359
 price      0.250759
 dtype: float64,
 carat      0.084091
 cut        0.163553
 color      0.321870
 clarity    0.104265
 depth      0.035806
 table      0.041187
 x          0.095559
 y          0.017318
 z          0.019982
 price      0.239243
 dtype: float64)

In [25]:
mod_data.describe()

,carat,cut,color,clarity,depth,table,x,y,z,price
count,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000,38433.000000
mean,0.114034,0.166825,0.378227,0.187455,0.521516,0.261805,0.524980,0.095783,0.109541,0.477252
std,0.084091,0.163553,0.321870,0.104265,0.035806,0.041187,0.095559,0.017318,0.019982,0.239243
min,0.004158,0.000000,0.000000,0.000000,0.355556,0.000000,0.358473,0.065365,0.000000,0.006658
25%,0.039501,0.000000,0.139749,0.141467,0.502778,0.235294,0.436685,0.079796,0.090881,0.257953
50%,0.103950,0.161548,0.453828,0.166956,0.525000,0.254902,0.526071,0.095925,0.109748,0.479162
75%,0.170478,0.198164,0.752440,0.266833,0.541667,0.294118,0.602421,0.109847,0.126101,0.671023
max,0.401247,1.000000,1.000000,1.000000,0.741667,0.509804,0.773743,0.140747,0.161321,1.000000
